In [3]:
# use with local installs that don't have nbdev
import sys

sys.path.insert(0, "../../")

In [4]:
from selenium.webdriver.common.by import By
import datacrew.crawler.crawler as dcc
import datacrew.crawler.article as dca


TEST_ARTICLE_URL = "https://domo-support.domo.com/s/topic/0TO5w000000ZanAGAS/beast-mode"
TEST_ARTICLE_URL = "https://domo-support.domo.com/s/article/360043429913"
TEST_TOPIC_URL = "https://domo-support.domo.com/s/topic/0TO5w000000ZamoGAC/creating-content-in-domo?language=en_US"

BASE_URL = "https://domo-support.domo.com"

IMG_BASE_URL = "https://domo-support.domo.com/servlet/rtaImage"

OUTPUT_FOLDER = "../../raw_kb"

driver = dcc.driversetup(is_headless=False)

test_article = dca.Article_KB(
    url=TEST_ARTICLE_URL, driver=driver, base_url=BASE_URL)

test_category = dca.Article_Category(
    url=TEST_TOPIC_URL, driver=driver, base_url=BASE_URL)



💤 retrieving https://domo-support.domo.com/s/article/360043429913 💤
💤 retrieving https://domo-support.domo.com/s/topic/0TO5w000000ZamoGAC/creating-content-in-domo?language=en_US 💤
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZanDGAS, 0TO5w000000ZanDGAS
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZaoEGAS, 0TO5w000000ZaoEGAS
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZaoNGAS, 0TO5w000000ZaoNGAS
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZanwGAC, 0TO5w000000ZanwGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZanZGAS, 0TO5w000000ZanZGAS
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000Zao4GAC, 0TO5w000000Zao4GAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZaoHGAS, 0TO5w000000ZaoHGAS
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZanAGAS, 0TO5w000000ZanAGAS
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZdTCGA0, 0TO5w000000

In [5]:
for key in test_article.__dict__.keys():
    if key in ['soup','article_soup']:
        continue

    # print({key: getattr(test_article, key)})

test_article.url_ls

['https://domo-support.domo.com/s/knowledge-base',
 'https://domo-support.domo.com/s/topic/0to5w000000zamwgac',
 'https://domo-support.domo.com/s/article/360042924034',
 'https://domo-support.domo.com/s/article/360043429933',
 'https://domo-support.domo.com/s/article/360043429953',
 'https://domo-support.domo.com/s/article/360042924094',
 'https://domo-support.domo.com/s/article/360043429993',
 'https://domo-support.domo.com/s/article/360043428753',
 'https://domo-support.domo.com/s/article/360043429913',
 'https://domo-support.domo.com/s/article/360042925494',
 'https://domo-support.domo.com/s/article/360043429693']

# Utility Functions

## string manipulation


In [6]:
import re

def convert_to_snake(text_str):
    """converts 'snake_case_str' to 'snakeCaseStr'"""

    return text_str.replace(" ", "_").lower()


def clean_url_name(path_name):
    valid_chars = r"[^a-zA-Z0-9_]"

    return re.sub(valid_chars, "", path_name)


def get_id_from_url(url: str, url_match: str):
    """use url_match" to identify the id of an object"""
    return url.split(url_match)[1].split("/")[0]


## process html files


### download image

In [7]:
import requests
import pandas as pd


def download_img(image_url, image_path, debug_prn: bool = False):

    img_data = requests.get(image_url).content

    with open(image_path, "wb") as handler:
        if debug_prn:
            print(f"downloading {image_url} to {image_path}")
        handler.write(img_data)


def download_article_images(
    article: dca.Article,
    output_path: str,
    debug_prn: bool = False,
):
    image_ls = article.image_ls

    if not image_ls:
        return 

    # download images
    for img in image_ls:
        img_name = img.get("name")

        if not img_name:
            continue

        img_url = img.get("url")
        img_path = f"{output_path}/{img_name}"
        img_rel_path = img.get("relative_url")

        if debug_prn:
            print(
                f"downloading {img_url} to {img_path}.  replacing article with {img_rel_path} with {img_name}"
            )

        download_img(image_url=img_url, image_path=img_path)

        article.md_str = article.md_str.replace(img_rel_path, img_name)

    return image_ls

test_output_path = "../../raw_kb/article/adding_a_beast_mode_calculation_to_your_chart"

pd.DataFrame(download_article_images(
    article=test_article,
    output_path=test_output_path,
    debug_prn=False,
))

,url,relative_url,name
0,https://domo-support.domo.com/servlet/rtaImage...,/servlet/rtaImage?eid=ka05w000001248e&feoid=00...,Beast_Mode_Dialog.jpg
1,https://domo-support.domo.com/servlet/rtaImage...,/servlet/rtaImage?eid=ka05w000001248e&feoid=00...,
2,https://domo-support.domo.com/servlet/rtaImage...,/servlet/rtaImage?eid=ka05w000001248e&feoid=00...,Beast_Mode_Modal.png
3,https://domo-support.domo.com/servlet/rtaImage...,/servlet/rtaImage?eid=ka05w000001248e&feoid=00...,
4,https://domo-support.domo.com/servlet/rtaImage...,/servlet/rtaImage?eid=ka05w000001248e&feoid=00...,


### download html

In [8]:
import os


def create_output_folder(output_folder, title: str):
    output_path = os.path.join(
        output_folder, clean_url_name(convert_to_snake(title))
    )

    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path


def output_html(output_html_path, soup):
    with open(output_html_path, 'w') as f:
        f.write(str(soup))


# handle category

In [9]:
import os
import pandas as pd
import datetime as dt

def update_listing(
        url: str,
        url_id: str,
        name: str,
        file_name: str,
        output_folder):

    output_file = f"{output_folder}/{file_name}"

    df = None
    if os.path.exists(output_file):
        df = pd.read_csv(output_file, index_col='id')

    else:
        columns = ['id','name','url', 'updated']
        df = pd.DataFrame(columns=columns).set_index('id')


   
    df.loc[url_id] = [name, url, dt.datetime.now()]

    df.to_csv(output_file)

    return df.loc[url_id]


update_listing(url=test_category.url,
                 url_id = test_category.url_id,
                 name = test_category.category,
                 file_name = 'category_mapping.csv',
                 output_folder=os.path.join(OUTPUT_FOLDER, 'category'))


name                                Creating Content In Domo
url        https://domo-support.domo.com/s/topic/0TO5w000...
updated                           2023-04-12 08:50:47.385049
Name: 0TO5w000000ZamoGAC, dtype: object

In [10]:
from mdutils.mdutils import MdUtils
import pandas as pd


def process_category(article: dca.Article,
                    output_folder: str,
                    debug_prn: bool = False):

    output_path = create_output_folder(os.path.join(output_folder,'category'), article.category)

    output_html_path = os.path.join(output_path, 'index.html')
    output_html(output_html_path, article.soup)

    update_listing(url=article.url,
                 url_id = article.url_id,
                 name = article.category,
                 file_name = 'category_listing.csv',
                 output_folder=OUTPUT_FOLDER)

    download_article_images(
        article=article,
        output_path= output_path)

process_category(test_category, output_folder=OUTPUT_FOLDER)


## handle article


In [11]:
from mdutils.mdutils import MdUtils
import pandas as pd


def process_article(article: dca.Article,
                    output_folder: str,
                    debug_prn: bool = False):

    output_path = create_output_folder(os.path.join(output_folder, 'article'), article.title)
    
    output_html(os.path.join(output_path, 'index.html'), article.soup)

    update_listing(url=article.url,
                url_id = article.url_id,
                name = article.title,
                file_name = 'article_listing.csv',
                output_folder=OUTPUT_FOLDER)

    download_article_images(
        article=article,
        output_path= output_path)

    

process_article(test_article, output_folder=OUTPUT_FOLDER)


In [12]:
import logging
import selenium.webdriver

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s", level=logging.INFO)


class Crawler:
    base_url: str
    output_folder: str

    urls_visited_ls: list[str]
    urls_to_vist_ls: list[str]
    urls_error_ls: list[str]

    path_to_visit: str
    path_errors : str
    path_visited: str

    driver: selenium.webdriver

    counter: int

    def __init__(
        self,
        urls_to_visit_ls: list[str] = None,
        base_url=None,
        output_folder="../../raw_kb/",
        is_fresh_start: bool = False
    ):
        self.base_url = base_url
        self.output_folder = output_folder

        self.counter = 0

        self.path_to_visit = os.path.join(
            self.output_folder, 'crawler_to_visit.csv')

        self.path_visited = os.path.join(
            self.output_folder, 'crawler_visited.csv')
        
        self.path_errors = os.path.join(self.output_folder, 'crawler_errors.csv')

        self.article_ls = []
        self.driver = dcc.driversetup(is_headless=False)

        if is_fresh_start:
            "✂️ deleting files"
            self._delete_file(self.path_to_visit)
            self._delete_file(self.path_visited)
            self._delete_file(self.path_errors)

        self.urls_visited_ls = []
        self.urls_to_visit_ls = []
        self.urls_error_ls =[]

        if not is_fresh_start:
            self.urls_visited_ls = self._read_file_ls(self.path_visited)

            urls_to_visit_ls += self._read_file_ls(self.path_errors, is_reverse = True)
            urls_to_visit_ls += self._read_file_ls(self.path_to_visit)

        [self.add_url_to_visit(url) for url in urls_to_visit_ls]

    @staticmethod
    def _delete_file(file_path):
        if os.path.exists(file_path):
            print(f'deleting {file_path}')
            os.remove(file_path)
        else:
            print(f"{file_path} cannot be deleted")

    @staticmethod
    def _read_file_ls(file_path, is_reverse: bool = False):
        try:
            file = open(file_path, '+r')

            if is_reverse:
                file.reverse()

            return [line.strip() for line in file]
        except:
            return []

    @staticmethod
    def _write_file_ls(file_path, data):

        file = open(file_path, 'w+')

        for item in data:
            file.write(item+"\n")
        file.close()

    def add_url_to_visit(self, url, debug_prn: bool = False):
        if url not in self.urls_visited_ls and url not in self.urls_to_visit_ls:
            if debug_prn:
                print(f"adding url to list - {url}")

            self.urls_to_visit_ls.append(url)

    def crawl(self, url, debug_prn: bool = False):
        if debug_prn:
            print(f"starting crawl - {url}")

        article = None
        if '/s/topic/' in url:
            article = dca.Article_Category(url=url,
                                           driver=driver,
                                           base_url=self.base_url,
                                           is_child_recursive=False
                                           )
            process_category(article=article, output_folder=self.output_folder)

        if '/s/article/' in url:
            article = dca.Article_KB(
                url=url, base_url=self.base_url, driver=driver)
            process_article(article=article, output_folder=self.output_folder)

        if not article:
            return

        for url in article.url_ls:
            self.add_url_to_visit(url=url, debug_prn=debug_prn)



    def run(self, debug_prn: bool = False):
        while self.urls_to_visit_ls:
            url = self.urls_to_visit_ls.pop(0)

            logging.info(f"Crawling: {url}")

            try:
                self.crawl(url, debug_prn)
                self.urls_visited_ls.append(url)
                
            except Exception:
                logging.exception(f"Failed to crawl: {url}")
                self.urls_error_ls.append(url)
            finally:
                self.counter += 1
                
                if self.counter % 10 == 0:
                    self._write_file_ls(self.path_to_visit, self.urls_to_visit_ls)
                    self._write_file_ls(self.path_visited, self.urls_visited_ls)
                    self._write_file_ls(self.path_errors, self.urls_error_ls)

        print("done")
        return self


NameError: name 'Bool' is not defined

In [ ]:
crawler = Crawler(
    urls_to_visit_ls=[TEST_ARTICLE_URL, TEST_TOPIC_URL], 
    base_url=BASE_URL, 
    output_folder= OUTPUT_FOLDER, 
    is_fresh_start = False
)

from pprint import pprint
pprint(crawler.__dict__)

crawler.run(debug_prn=False)

2023-04-12 07:29:57,510 INFO:Crawling: https://domo-support.domo.com/s/topic/0to5w000000zanagas


{'article_ls': [],
 'base_url': 'https://domo-support.domo.com',
 'counter': 0,
 'driver': <selenium.webdriver.chrome.webdriver.WebDriver (session="ceeb413e3086db41a13649323ada9eb7")>,
 'output_folder': '../../raw_kb',
 'path_errors': '../../raw_kb\\crawler_errors.csv',
 'path_to_visit': '../../raw_kb\\crawler_to_visit.csv',
 'path_visited': '../../raw_kb\\crawler_visited.csv',
 'urls_error_ls': [],
 'urls_to_visit_ls': ['https://domo-support.domo.com/s/topic/0to5w000000zanagas',
                      'https://domo-support.domo.com/s/topic/0to5w000000zdtcga0',
                      'https://domo-support.domo.com/s/article/360042923894',
                      'https://domo-support.domo.com/s/topic/0to5w000000zancgas',
                      'https://domo-support.domo.com/s/article/360042935334',
                      'https://domo-support.domo.com/s/article/360043427633',
                      'https://domo-support.domo.com/s/article/360043430673',
                      'https://domo-sup

2023-04-12 07:30:14,748 ERROR:Failed to crawl: https://domo-support.domo.com/s/topic/0to5w000000zanagas
Traceback (most recent call last):
  File "C:\Users\jwilson1\AppData\Local\Temp\1\ipykernel_24624\2488320220.py", line 130, in run
    self.crawl(url, debug_prn)
  File "C:\Users\jwilson1\AppData\Local\Temp\1\ipykernel_24624\2488320220.py", line 103, in crawl
    article = dca.Article_Category(url=url,
  File "c:\Users\jwilson1\GitHub\datacrew\nbs\projects\../..\datacrew\crawler\article.py", line 287, in __init__
    raise ArticleKB_GetSoupError(url=url)
datacrew.crawler.article.ArticleKB_GetSoupError: failed to retrieve soup for https://domo-support.domo.com/s/topic/0to5w000000zanagas
2023-04-12 07:30:14,750 INFO:Crawling: https://domo-support.domo.com/s/topic/0to5w000000zdtcga0


Message: 
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0115DCE3+50899]
	(No symbol) [0x010EE111]
	(No symbol) [0x00FF5588]
	(No symbol) [0x010208F9]
	(No symbol) [0x01020AFB]
	(No symbol) [0x0104F902]
	(No symbol) [0x0103B944]
	(No symbol) [0x0104E01C]
	(No symbol) [0x0103B6F6]
	(No symbol) [0x01017708]
	(No symbol) [0x0101886D]
	GetHandleVerifier [0x013C3EAE+2566302]
	GetHandleVerifier [0x013F92B1+2784417]
	GetHandleVerifier [0x013F327C+2759788]
	GetHandleVerifier [0x011F5740+672048]
	(No symbol) [0x010F8872]
	(No symbol) [0x010F41C8]
	(No symbol) [0x010F42AB]
	(No symbol) [0x010E71B7]
	BaseThreadInitThunk [0x76CB00F9+25]
	RtlGetAppContainerNamedObjectPath [0x775D7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x775D7B8E+238]

name 'url' is not defined
💤 retrieving https://domo-support.domo.com/s/topic/0to5w000000zdtcga0 💤


2023-04-12 07:30:30,935 ERROR:Failed to crawl: https://domo-support.domo.com/s/topic/0to5w000000zdtcga0
Traceback (most recent call last):
  File "C:\Users\jwilson1\AppData\Local\Temp\1\ipykernel_24624\2488320220.py", line 130, in run
    self.crawl(url, debug_prn)
  File "C:\Users\jwilson1\AppData\Local\Temp\1\ipykernel_24624\2488320220.py", line 103, in crawl
    article = dca.Article_Category(url=url,
  File "c:\Users\jwilson1\GitHub\datacrew\nbs\projects\../..\datacrew\crawler\article.py", line 287, in __init__
    raise ArticleKB_GetSoupError(url=url)
datacrew.crawler.article.ArticleKB_GetSoupError: failed to retrieve soup for https://domo-support.domo.com/s/topic/0to5w000000zdtcga0
2023-04-12 07:30:30,937 INFO:Crawling: https://domo-support.domo.com/s/article/360042923894


...

💤 retrieving https://domo-support.domo.com/s/article/360043428913 💤


2023-04-12 08:40:50,325 INFO:Crawling: https://domo-support.domo.com/s/article/360042924494


💤 retrieving https://domo-support.domo.com/s/article/360042924494 💤


2023-04-12 08:41:10,063 INFO:Crawling: https://domo-support.domo.com/s/article/360042925054


💤 retrieving https://domo-support.domo.com/s/article/360042925054 💤
done
